# 6D Pose Estimation

## Set up the project

We will work with a portion of this dataset, which you can find here: https://drive.google.com/drive/folders/19ivHpaKm9dOrr12fzC8IDFczWRPFxho7

Set some variables to conditionally run some codes. First download the project and change directory to ```6DPose_Estimation```

In [ ]:
MOUNT_DRIVE = False
COMET_ML = False

In [ ]:
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd /content/drive/MyDrive/6DPose_Estimation/

Install all dependencies of PyTorch dependencies

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
%%capture
import os
import torch

%env TORCH=$torch.__version__
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-${TORCH}.html

Install all packages, you may need to restart the runtime before continuing

In [ ]:
!pip install -r ./requirements.txt
print("Restart runtime")

In [ ]:
import os
import yaml
import torch
import torchvision
import open3d as o3d
import itertools
import shutil
import ultralytics
from torch.utils.data import Dataset
from torch import nn, optim
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import matplotlib.patches as patches
import wandb
from scipy.spatial.transform import Rotation as R
from torchvision import models
import cv2
from torch.optim import Adam
import quaternion
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from ultralytics import YOLO
from torchvision.transforms import v2
import trimesh

# install PyTorch Geometric after installation and restart
import torch_geometric
from torch import Tensor
from torch_geometric.nn import knn_interpolate, MessagePassing
from torch_geometric.nn.pool import fps, radius

# import comet-ml
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import watch

from utils.data_exploration import load_image
from utils.installation_checker import check_torch_geometric

from data.CustomDatasetPose import IMG_WIDTH, IMG_HEIGHT

# check if everything works
check_torch_geometric()

Set device

In [ ]:
from utils.device_setter import set_device

device = set_device()

## Download dataset

In [ ]:
# Step 1: Download the dataset (LineMOD)
# Download LineMOD dataset
# create directory structure without errors
!mkdir -p datasets/linemod/
%cd datasets/linemod/

In [ ]:
!mkdir -p DenseFusion/
%cd DenseFusion/

In [ ]:
# Download dataset (which includes a portion of the LimeMOD dataset)
!gdown --folder "https://drive.google.com/drive/folders/19ivHpaKm9dOrr12fzC8IDFczWRPFxho7"

In [ ]:
if MOUNT_DRIVE:
    !cp /content/drive/MyDrive/6DPose_Estimation /content/ # move to content for faster access to files
    %cd /content/6DPose_Estimation/datasets/linemod/DenseFusion

!unzip Linemod_preprocessed.zip
!rm Linemod_preprocessed.zip
%cd ../../../ # change directory to 6D_pose_estimation

Get working directory

In [ ]:
path = !pwd
path = path[0]

## Modify Dataset

Copy ground truth files to ```Linemod_preprocessed```

In [ ]:
from utils.data_exploration import get_class_names
from utils.preprocessing import copy_gt_file, change_02gt, quaternion_gt

folder_names = get_class_names()
copy_gt_file(folder_names)

Change ```02_gt.yml``` to take only one object

In [ ]:
change_02gt("./datasets/linemod/DenseFusion/Linemod_preprocessed/02_gt.yml")

Add quaternion

In [ ]:
quaternion_gt("./datasets/linemod/DenseFusion/Linemod_preprocessed")

## Data Exploration

Load an image

In [ ]:
load_image(label=1, object=0)

Check if camera intrinsics is same for all images

In [ ]:
root_path = "./datasets/linemod/DenseFusion/Linemod_preprocessed/data"

from utils.data_exploration import check_cam_K_equal

cam_K = check_cam_K_equal(root_path)

## Define CustomDataset

In [ ]:
from data.CustomDatasetPose import CustomDatasetPose

dataset_root = "./datasets/linemod/DenseFusion/Linemod_preprocessed/"

train_dataset = CustomDatasetPose(dataset_root, split="train", device=device, cam_K = cam_K)
print(f"Training samples: {len(train_dataset)}")

val_dataset = CustomDatasetPose(dataset_root, split="validation", device=device, cam_K = cam_K)
print(f"Validation samples: {len(val_dataset)}")

test_dataset = CustomDatasetPose(dataset_root, split="test", device=device, cam_K = cam_K)
print(f"Testing samples: {len(test_dataset)}")

## Data Preprocessing

Structure the data for YOLO such that
```
datasets/
├── data.yaml
│
├── train/
│   ├── images/
│   │
│   └── labels/
│  
├── val/
│
└── test/
```

In [ ]:
# divide the dataset into training, validation and testing set
train_samples = train_dataset.get_samples_id()
validation_samples = val_dataset.get_samples_id()
test_samples = test_dataset.get_samples_id() # test folder is optional for training YOLO

Create a new folder containing all the info, we just need the rgb image and a text file with the label and bounding box.
The ```Linemod_preprocessed``` is not removed, as it contains info about translation and rotation that are needed for pose estimation, but not for object detection model.

The working directory is in the ```6DPose_Estimation```

Create YOLO yaml

In [ ]:
from utils.preprocessing import create_YOLO_yaml, create_dataset_YOLO

number_classes, class_names = create_YOLO_yaml(path, folder_names)

While creating the folder structure, we have to change the class id by using the index in the array written in the ```data.yaml```

In [ ]:
# create a dictionary to have easily access to the index
index_dict = dict()
for index, el in enumerate(class_names):
    index_dict[int(el)] = index

Create the folders. Note that each image may contain multiple objects. For instance in ```data/02/gt.yml``` for one image there are multiple objects, but just consider the object of that class

In [ ]:
counter_df = create_dataset_YOLO(number_classes, train_samples, validation_samples, test_samples, index_dict, path, train_dataset)

Visualize dataset distribution

In [ ]:
from utils.data_exploration import load_dataset_distribution

load_dataset_distribution(counter_df, index_dict, number_classes)

### Visualize data

Visualize depth image

In [ ]:
from utils.data_exploration import load_depth_image

folder = "02"
object_name = "0101"
img = load_depth_image(f"{path}/datasets/linemod/DenseFusion/Linemod_preprocessed/data/{folder}/depth/{object_name}.png")

Plot the patch of first object of the image, it reads from the ground truth file containing also multiple objects in one image

In [ ]:
from utils.data_exploration import load_depth_patch

load_depth_patch(path, folder, object_name, img)

Get data loader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
print(f"Training loader: {len(train_loader)}")
print(f"Validation loader: {len(val_loader)}")
print(f"Test loader: {len(test_loader)}")

Plot one batch of data

In [ ]:
from utils.data_exploration import plot_batch_data

plot_batch_data(train_loader, val_loader, test_loader)

## Training Object Detection model

In [ ]:
from train_YOLO import train_YOLO

epochs = 50
batch_size = 64
IMG_SIZE = 640

train_YOLO(path, epochs, batch_size, device, IMG_SIZE) # train model and save it to checkpoints

Validate model on test set

In [ ]:
from evaluate_YOLO import evaluate_YOLO

evaluate_YOLO(path, epochs, batch_size, IMG_SIZE, device)

## Pose Estimator Module

Plot

In [ ]:
import quaternion
import trimesh

In [ ]:
def plotPose(pathImage, translation_gt, rotation_gt, translation_pred, rotation_pred):
    '''
        Input:
            path for image (in DenseFusion)
            ground truth translation tensor (in millimeters)
            ground truth rotation tensor (either matrix or quaternion)
            predicted translation tensor (in millimeters)
            predicted rotation tensor (either matrix or quaternion)
    '''

    # read image
    image = cv2.imread(pathImage)
    transparent_image = image.copy() # copy of the image to work on a transparent image (for the second reference system)

    # read translation and rotation
    rotat_gt = rotation_gt.numpy() # transform tensor to numpy array
    trans_gt = translation_gt.numpy()/1000 # in meters
    rotat_pred = rotation_pred.numpy()
    trans_pred = translation_pred.numpy()/1000

    # read camera intrinsics
    label = pathImage.split("/")[-1].split(".")[0].split("_")[0]
    image_id = pathImage.split("/")[-1].split(".")[0].split("_")[1]
    with open(f"./YOLO/datasets_cropped/{label}_info.yml", 'r') as f:
            camera_info = yaml.load(f, Loader=yaml.FullLoader)
    camera_intrinsics = np.array(camera_info[int(image_id)]["cam_K"]).reshape(3,3)

    # read 3D model
    meshModel = trimesh.load(f"./DenseFusion/Linemod_preprocessed/models/obj_{label}.ply")
    vertices = meshModel.vertices/1000 # it has 3 columns, for X, Y, Z, use unit of measurement of translation
    # compute corners
    min_corner = vertices.min(axis=0) # find for each column the smallest value
    max_corner = vertices.max(axis=0)
    
    bounding_box_3d = np.array([[min_corner[0], min_corner[1], min_corner[2]],
                                [max_corner[0], min_corner[1], min_corner[2]],
                                [max_corner[0], max_corner[1], min_corner[2]],
                                [min_corner[0], max_corner[1], min_corner[2]],
                                [min_corner[0], min_corner[1], max_corner[2]],
                                [max_corner[0], min_corner[1], max_corner[2]],
                                [max_corner[0], max_corner[1], max_corner[2]],
                                [min_corner[0], max_corner[1], max_corner[2]],])

    # convert quaternion to rotation matrix, if input was quaternion
    if rotat_gt.size == 4:
        rotat_gt = quaternion.as_rotation_matrix(np.quaternion(*rotat_gt))
    else:
        rotat_gt = rotat_gt.reshape(3,3)
    if rotat_pred.size == 4:
        rotat_pred = quaternion.as_rotation_matrix(np.quaternion(*rotat_pred))
    else:
        rotat_pred = rotat_pred.reshape(3,3)

    # build 3D axes according to object coordinate system, same unit of measurement of translation, so in meters
    axes_3d = np.array([
        [0, 0, 0],      # origin, in the object coordinate system
        [0.15, 0, 0],   # how long the arrow should be in the X coordinate
        [0, 0.15, 0],   # how long the arrow should be in the Y coordinate
        [0, 0, 0.15]    # how long the arrow should be in the Z coordinate
    ])

    # transform the object coordinate system to the camera coordinate system
    # rotat_gt is 3x3, so axes_3d has to be transposed, then add to origin, and coordinates the translation
    axes_cam_gt = (rotat_gt @ axes_3d.T).T + trans_gt
    # bounding box
    bounding_box_3d_cam_gt = (rotat_gt @ bounding_box_3d.T).T + trans_gt
    # project 3D axes to 2D
    axes_2d_gt = (camera_intrinsics @ axes_cam_gt.T).T # camera_intrinsics is 3x3, while axes_cam_gt 4x3, axes_2d_gt 4x3
    axes_2d_gt = axes_2d_gt[:, :2] / axes_2d_gt[:, 2:3] # take first 2 columns and normalize by depth
    # bounding box
    bounding_box_2d_gt = (camera_intrinsics @ bounding_box_3d_cam_gt.T).T
    bounding_box_2d_gt = (bounding_box_2d_gt[:, :2] / bounding_box_2d_gt[:, 2:3]).astype(int)
    # get point coordinates
    p_gt = [tuple(el) for el in bounding_box_2d_gt]
    # define edges using two points, access with index
    edges = [(0,1), (1,2), (2,3), (3,0), # bottom
             (0,4), (1,5), (2,6), (3,7), # vertical
             (4,5), (5,6), (6,7), (7,4) # top
             ]
    # draw edges
    for el in edges:
        cv2.line(image, p_gt[el[0]], p_gt[el[1]], (0,0,255), 5)

    p0_gt = tuple(axes_2d_gt[0].astype(int)) # take origin coordinates
    p1_gt = tuple(axes_2d_gt[1].astype(int))
    p2_gt = tuple(axes_2d_gt[2].astype(int))
    p3_gt = tuple(axes_2d_gt[3].astype(int))

    # color is in BGR format, set tickness=2
    cv2.arrowedLine(image, p0_gt, p1_gt, (0, 0, 255), 2) # X is red
    cv2.arrowedLine(image, p0_gt, p2_gt, (0, 255, 0), 2) # Y is green
    cv2.arrowedLine(image, p0_gt, p3_gt, (255, 0, 0), 2) # Z is blue

    # for predicted
    axes_cam_pred = (rotat_pred @ axes_3d.T).T + trans_pred
    bounding_box_3d_cam_pred = (rotat_pred @ bounding_box_3d.T).T + trans_pred

    axes_2d_pred = (camera_intrinsics @ axes_cam_pred.T).T
    axes_2d_pred = axes_2d_pred[:, :2] / axes_2d_pred[:, 2:3]
    bounding_box_2d_pred = (camera_intrinsics @ bounding_box_3d_cam_pred.T).T
    bounding_box_2d_pred = (bounding_box_2d_pred[:, :2] / bounding_box_2d_pred[:, 2:3]).astype(int)

    p_pred = [tuple(el) for el in bounding_box_2d_pred]
    edges = [(0,1), (1,2), (2,3), (3,0), # bottom
             (0,4), (1,5), (2,6), (3,7), # vertical
             (4,5), (5,6), (6,7), (7,4) # top
             ]
    for el in edges:
        cv2.line(image, p_pred[el[0]], p_pred[el[1]], (255,0,0), 5)

    p0_pred = tuple(axes_2d_pred[0].astype(int))
    p1_pred = tuple(axes_2d_pred[1].astype(int))
    p2_pred = tuple(axes_2d_pred[2].astype(int))
    p3_pred = tuple(axes_2d_pred[3].astype(int))

    cv2.arrowedLine(transparent_image, p0_pred, p1_pred, (0, 0, 255), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p2_pred, (0, 255, 0), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p3_pred, (255, 0, 0), 2)

    # show image after merging the two images
    overlapImage = cv2.addWeighted(transparent_image, 0.5, image, 1, 0)
    plt.imshow(cv2.cvtColor(overlapImage, cv2.COLOR_BGR2RGB))
    plt.title("Object Pose Estimation (prediction is transparent)")
    plt.show()

    return